In [17]:
# 라이브러리 import
import os
import pandas as pd
import numpy as np
import math 

# 셀레늄
from selenium.webdriver.common.alert import Alert
from selenium import webdriver  # 라이브러리(모듈) 가져오라
# pip install chromedriver_autoinstaller
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC

from selenium.webdriver.chrome.service import Service as ChromeService

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# for문으로 크롤링 

In [5]:
# 검색어 입력
search = '삼성전자'

# 크롬 옵션
options = webdriver.ChromeOptions()

# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=1000,1200") # window-size -> 기본 : 1920,1080

import chromedriver_autoinstaller

chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)
time.sleep(0.01)
driver.get("https://www.naver.com") # -> 네이버 크롬 창이 뜬다.

# 네이버 검색어 입력 후 검색
element = driver.find_element("name", "query") 
element.clear()  # 혹시 검색창에 존재하는 텍스트 제거

element.send_keys(search)  # 검색창에 검색어 전달
element.submit()  # 검색 클릭
time.sleep(0.01)

In [6]:
# 뉴스 클릭
driver.find_element("link text", "뉴스").click() 
time.sleep(0.01)

In [7]:
# '오래된 순' 클릭
driver.find_element("link text", "오래된순").click()  
time.sleep(0.01)

In [93]:
BASE_URL = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sort=2&photo=0&field=0&pd=3&"
# https://finance.naver.com/sise/sise_market_sum.naver?&page=1
start = '1990.01.01'
end = '1990.02.01'
search_span= "ds=" + start + "&de=" + end + "&a:all&start=" # 기본값


In [94]:
# 뉴스 데이터 수집
news_list = []
date_list = []
title_list = []

BASE_URL = BASE_URL + search_span

service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(BASE_URL+"1")

for i in range(5):  
    
    # 언론사
    newses = driver.find_elements('css selector', 'a.info.press')
    temp_list = []
    for news in newses:
        temp_list.append(news.text)
#         temp_list = [x for x in temp_list ]
    news_list = news_list + temp_list 
    
    # 날짜
    dates = driver.find_elements('css selector', 'span.info')
    temp_list = []
    for date in dates:
        temp_list.append(date.text)
#         temp_list = [x for x in temp_list ]
    date_list = date_list + temp_list 
    
    # 제목
    titles = driver.find_elements('css selector', 'a.news_tit')
    temp_list = []
    for title in titles:
        temp_list.append(title.text)
#         temp_list = [x for x in temp_list ]
    title_list = title_list + temp_list 
    
    # 다음페이지 클릭, next 비활성화 되면 멈춤
    if driver.find_element('css selector', 'a.btn_next').get_attribute('aria-disabled') == 'false':
        driver.find_element('css selector', 'a.btn_next').click()
        continue;
    elif (driver.find_element('css selector', 'a.btn_next').get_attribute('aria-disabled') == 'true') & (driver.find_element('css selector', 'a.btn').get_attribute('name') < 400) : # 추출된 페이지 수가 400보다 작으면
        break;
    elif driver.find_element('css selector', 'a.btn_next').get_attribute('aria-disabled') == 'true':
        # 날짜를 마지막 뉴스 날짜로 업데이트
        search_span = "ds=" + date_list[-1] + "&de=" + end + "&a:all&start=" # 시작, 끝 날짜 설정
        
        BASE_URL = BASE_URL + search_span
        service = ChromeService(executable_path=ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service)
        driver.get(BASE_URL+"1")
        continue;

#     elif driver.find_element('css selector', 'div.not_fount02') # 검색결과 없음 :
#         break;
        
#     driver.find_element('css selector', 'a.btn_next').click()

    time.sleep(0.001)
    

print(len(title_list))
# print(news_list)
print(date_list)
# print(title_list)

TypeError: '<' not supported between instances of 'str' and 'int'

In [67]:
print(date_list[-1])

1990.02.01.


In [71]:
a = 'f'
b = 's'
a+b

'fs'

In [70]:
x = "ds=" + date_list[-1] + "&de=1990.02.01&a:all&start="
type(x)

str

In [59]:
# df로 만들기

df = pd.DataFrame(list(zip(news_list, date_list, title_list)), columns = ['언론사','날짜', '제목'])

df

# csv로 뽑기
# df.to_csv('.csv')

,언론사,날짜,제목
0,연합뉴스,1990.01.17.,朴守敎-林正明 코트서 벤치 대결로
1,연합뉴스,1990.01.17.,"EC, 한국산 CDP 반덤핑 확정관세 결정"
2,연합뉴스,1990.01.17.,범용 CAD 개발
3,연합뉴스,1990.01.19.,"광고업계,제조업 불황속 호황 누려"
4,연합뉴스,1990.01.21.,-농구대잔치-
5,연합뉴스,1990.01.21.,-농구대잔치 2차대회 중간결산-
6,연합뉴스,1990.01.23.,농구대잔치 남녀 새스타 3명 배출
7,연합뉴스,1990.01.24.,일부 전기주전자 화재발생 위험
8,연합뉴스,1990.01.24.,"美, 한국산 사설전화교환기 덤핑관세 부과"
9,연합뉴스,1990.01.24.,-농구대잔치-


In [54]:
for i in range(5):
    print(i)

0
1
2
3
4
